**Classificação - Mineração de Dados**


**Nome: Davi Augusto Neves Leite**

**Data de Entrega: 10/10/2023**


---


# <strong> Materiais </strong>


Os principais recursos para a execução desta atividade podem ser vistos a seguir.

1. <strong>Software</strong>

- Sistemas Operacionais: Windows 11 para _desktop_;
- Ambiente de Desenvolvimento Integrado: Microsoft Visual Studio Code;
- Linguagem de Programação: Python 3.11.5 64-bit.

2. <strong>Hardware</strong>

- Notebook pessoal Lenovo Ideapad 330-15IKB com: processador Intel Core i7-8550U, HDD WD Blue WD10SPZX de 1TB, SSD Crucial BX500 de 240GB, 12 GB DDR4 de Memória RAM e placa de vídeo NVIDIA GeForce MX150 (2 GB GDDR5 de memória).


---


# <strong> Importação das Bibliotecas Principais </strong>

Nota: ao decorrer deste Notebook, outras bibliotecas podem ser utilizadas em quaisquer respectiva seção/conjunto de dados, dependendo da necessidade. Abaixo, há a importação das principais que são comuns e utilizadas em todas ou quase todas seções/conjunto de dados.


In [372]:
import numpy as np      # Manipulação de listas
import pandas as pd     # Manipulação de tabelas
import seaborn as sbn   # Geração de gráficos estatísticos
import matplotlib.pyplot as plt  # Geração de gráficos de listas
import sklearn as skl   # Biblioteca para pré-processamento
from copy import copy as cp  # Possibilitar copiar os objetos

# Ignorar os avisos não importantes durante a execução deste notebook
import warnings
warnings.filterwarnings('ignore')


---
